In [177]:
from astropy.coordinates import Angle
from astropy.table import QTable, vstack
import astropy.units as u

In [135]:
with open("wdsweb_summ2.txt", "r", newline="\n") as f:
    for _ in range(5):
        f.readline()
    rows=[]
    for line in f.readlines():
        try:
            row = {}
            row["ID"] = line[0:10]
            row["Separation"] = (float(line[46:51]) + float(line[51:57])) / 2 * u.arcsec
            row["Mag A"] = float(line[58:63])
            row["Mag B"] = float(line[63:69])
            row["Mag Avg"] = (row["Mag A"] + row["Mag B"]) / 2
            row["Spectral Type"] = line[70:79]
            row["RA"] = Angle(f"{line[112:114]}h{line[114:116]}m{line[116:121]}s")
            row["Dec"] = Angle(f"{line[121:124]}d{line[124:126]}m{line[126:130]}s")
            rows.append(row)
        except:
            continue
wds = QTable(rows)
wds.write("wdsweb_summ2.ecsv", overwrite=True)
wds

In [180]:
# wds["Sep Category"] = (wds["Separation"].value / 10).astype(int) * 10

criteria = (wds["Separation"] > 45 * u.arcsec) & (wds["Separation"] < 4 * u.arcmin) & (wds["Mag Avg"] < 9)

filtered = wds[criteria]
filtered["Sep Category"] = (filtered["Separation"].value / 10).astype(int) * 10
filtered.sort(["Mag Avg"])
# filtered.sort(["Sep Category", "Mag Avg"])
filtered

target_list = vstack([group[0:5] for group in filtered.group_by("Sep Category").groups])
target_list


ID,Separation,Mag A,Mag B,Mag Avg,Spectral Type,RA,Dec,Sep Category
,arcsec,,,,,hourangle,deg,
str10,float64,float64,float64,float64,str9,float64,float64,int32
18501+3322,46.75,3.63,6.69,5.16,B7Ve+A8p,18.834663888888887,33.36266666666667,40
05354-0555,48.8,2.77,9.81,6.29,+A0V,5.5905499999999995,-5.909888888888889,40
01144-0755,48.7,5.19,7.85,6.52,F6V+G9V,1.2400111111111112,-7.922833333333334,40
18138-2104,45.1,3.85,9.22,6.535,B8Iape,18.229391666666665,-21.058833333333332,40
10314-5343,47.95,4.95,8.58,6.765000000000001,F6V,10.522727777777778,-53.715472222222225,40
05320-0018,50.9,2.41,6.83,4.62,O9.5II+,5.533444444444444,-0.2990833333333333,50
10393-5536,52.55,4.38,6.06,5.22,G2-3Ib,10.655108333333335,-55.60325,50
05354-0525,52.55,5.03,6.19,5.61,O9.5IV+B3,5.5896888888888885,-5.416027777777778,50
